In [1]:
import glob
import pandas as pd

listOfFileNames = []
listOfClassNames = []
listOfDataPerClass = []
files = []
classNames = []
#path = "/Users/user/Desktop/asd/thesis/placeholder/"
#path = "/Users/user/Desktop/pinakalatest/thesis/placeholder/"
path = "C:/Users/user/Desktop/Data/nonsignerdatalatest/non-signer_data/"
classNames = glob.glob(path + "*")

for c in classNames:
    className = c.split("\\")[1]
    listOfClassNames.append(c.split("\\")[1])
    files = glob.glob(path + className+"/*") 
    listOfFileNames = []
    for i in range(0, len(files)):
        data = glob.glob(path + className+"/" + str(i+1) + "/*.csv")
        
        for f in data:
            listOfFileNames.append(str(i+1) + "/" + f.split("\\")[1])
    listOfDataPerClass.append(listOfFileNames)
    
listOfDataFramesPerClass = {}
dataFrameDictionary = {}
dataFrames = []

for c, d in zip(listOfClassNames, listOfDataPerClass):
    print("----------------- Word " + c + "-----------------")
    emgDataFrames = {}
    emgDataFrames['right'] = []
    emgDataFrames['left'] = []
    
    accDataFrames = {}
    accDataFrames['right'] = []
    accDataFrames['left'] = []

    gyroDataFrames = {}
    gyroDataFrames['right'] = []
    gyroDataFrames['left'] = []

    dataFrameDictionary = {}
    className = c
    emgleftctr = 0
    emgrightctr = 0
    accleftctr = 0
    accrightctr = 0
    gyroleftctr = 0
    gyrorightctr = 0
    
    for fileName in d:
        fileLocation = path + className + "/" + fileName + ""
            
        df = pd.read_csv(fileLocation)
            
        if "emg" in fileName:   
            if "-0-" in fileName:
                emgDataFrames['right'].append(df)
                emgleftctr+=1
            else:
                emgDataFrames['left'].append(df)
                emgrightctr+=1
        elif ("accelerometer") in fileName:
            if "-0-" in fileName:
                accDataFrames['right'].append(df)
                accleftctr+=1
            else:
                accDataFrames['left'].append(df)
                accrightctr+=1
        elif ("gyro") in fileName:
            if "-0-" in fileName:
                gyroDataFrames['right'].append(df)
                gyroleftctr+=1
            else:
                gyroDataFrames['left'].append(df)
                gyrorightctr+=1
    
    dataFrameDictionary["emg"] = emgDataFrames
    dataFrameDictionary["acc"] = accDataFrames
    dataFrameDictionary["gyro"] = gyroDataFrames

    listOfDataFramesPerClass[className] = dataFrameDictionary
    print("Total of " + str(emgleftctr) + " left arm emg files and " + str(emgrightctr) + " right arm emg files.")
    print("Total of " + str(accleftctr) + " left arm acc files and " + str(accrightctr) + " right arm acc files.")
    print("Total of " + str(gyroleftctr) + " left arm gyro files and " + str(gyrorightctr) + " right arm gyro files.")
    
print("DONE!")

----------------- Word ano-----------------
Total of 100 left arm emg files and 100 right arm emg files.
Total of 100 left arm acc files and 100 right arm acc files.
Total of 100 left arm gyro files and 100 right arm gyro files.
----------------- Word asul-----------------
Total of 100 left arm emg files and 100 right arm emg files.
Total of 100 left arm acc files and 100 right arm acc files.
Total of 100 left arm gyro files and 100 right arm gyro files.
----------------- Word ate-----------------
Total of 100 left arm emg files and 100 right arm emg files.
Total of 100 left arm acc files and 100 right arm acc files.
Total of 100 left arm gyro files and 100 right arm gyro files.
----------------- Word baboy-----------------
Total of 100 left arm emg files and 100 right arm emg files.
Total of 100 left arm acc files and 100 right arm acc files.
Total of 100 left arm gyro files and 100 right arm gyro files.
----------------- Word dalawa-----------------
Total of 100 left arm emg files an

In [5]:
import numpy as np
listOfTrimmedDataFramesPerClass = {}

for className in classNames:
    className = className.split("\\")[1]
    print("----------------- Word " + className + "-----------------")
    ctr = 0
    trimmedDataFramesPerClass = {}
    trimmedDataFramesPerClass['left'] = []
    trimmedDataFramesPerClass['right'] = []
    
    dataFrameDictionary = {}
    
    emgDataFrames = listOfDataFramesPerClass[className]['emg']
    accDataFrames = listOfDataFramesPerClass[className]['acc']
    gyroDataFrames = listOfDataFramesPerClass[className]['gyro']
    
    trimmedAccDataFrames = []
    trimmedGyroDataFrames = []
    trimmedEmgDataFrames = []

    
    for i in range(0,len(listOfDataFramesPerClass[className]['acc']['right'])):
        start = -1
        end = -1
        counter = 0
        print("---- %s ----" % i)
        ctr += 1
        dataFrame = listOfDataFramesPerClass[className]['acc']['right'][i]
        for x, z in zip(dataFrame['x'], dataFrame['z']):  
            noend = False
            isBreak = False
            if(className in ("isda", "magkano")):
                if(x < 0.85 and start ==-1):
                    #print(x)
                    start = counter - 10
                    if start < 0:
                        start = 0
                elif(start != -1 and x>0.85):                   
                    end = counter + 10
                    print("rows: " + str(np.abs(start-end)))
                    if(np.abs(start-end) < 70):
                        end = -1
                    else:
                        break;
            else:
                if(x < z and start ==-1):
                    #print(x)
                    start = counter - 10
                    if start < 0:
                        start = 0
                elif(start != -1 and x>0.85):
                    end = counter + 10
                    print("rows: " + str(np.abs(start-end)))
                    if(np.abs(start-end) < 70):
                        end = -1
                    else:
                        break;
                        
            noend = True                
            counter += 1
            
        if noend == True or end >= len(listOfDataFramesPerClass[className]['acc']['right'][i]['x']):
            print(" NO END ey" )
            end = len(listOfDataFramesPerClass[className]['acc']['right'][i]['x'])-1
        

        startEmg = -1
        endEmg = -1
        counter2 = 0
        
        #print("Start index: %s End index: %s" % (start, end))
        for t in emgDataFrames['right'][i]['timestamp']:
            noend = False
            if(t > accDataFrames['right'][i].loc[start]['timestamp'] and startEmg == -1):
                #print("%s > %f" % (t, accDataFrames['right'][i].loc[start]['timestamp']))
                startEmg = counter2
            elif(startEmg != -1 and t > accDataFrames['right'][i].loc[end]['timestamp']):
                #print("%s > %f" % (t, accDataFrames['right'][i].loc[end]['timestamp']))
                endEmg = counter2
                break;
            else:
                noend = True
            counter2 += 1
        
        if noend == True:
       #     print(" NO END")
            endEmg = len(listOfDataFramesPerClass[className]['emg']['right'][i]['timestamp'])-1
        
        newAccDf = {}
        newAccDf['right'] = accDataFrames['right'][i].loc[start:end]
        newAccDf['right'] = newAccDf['right'].reset_index(drop=True)
        newAccDf['left'] = accDataFrames['left'][i].loc[start:end]
        newAccDf['left'] = newAccDf['left'].reset_index(drop=True)
        
        newGyroDf = {}
        newGyroDf['right'] = gyroDataFrames['right'][i].loc[start:end]
        newGyroDf['right'] = newGyroDf['right'].reset_index(drop=True)
        newGyroDf['left'] = gyroDataFrames['left'][i].loc[start:end]
        newGyroDf['left'] = newGyroDf['left'].reset_index(drop=True)
        
        newEmgDf = {}
        newEmgDf['right'] = emgDataFrames['right'][i].loc[startEmg:endEmg]
        newEmgDf['right'] = newEmgDf['right'].reset_index(drop=True)
        newEmgDf['left'] = emgDataFrames['left'][i].loc[startEmg:endEmg]
        newEmgDf['left'] = newEmgDf['left'].reset_index(drop=True)

        trimmedAccDataFrames.append(newAccDf)
        trimmedGyroDataFrames.append(newGyroDf)
        trimmedEmgDataFrames.append(newEmgDf)
        
        #print("Start emg index: %s End emg index: %s" % (startEmg, endEmg))
        
    dataFrameDictionary['acc'] = trimmedAccDataFrames
    dataFrameDictionary['gyro'] = trimmedGyroDataFrames
    dataFrameDictionary['emg'] = trimmedEmgDataFrames
        
    listOfTrimmedDataFramesPerClass[className] = dataFrameDictionary

        
    
    print("Done Trimming " + className + " " + str(ctr) + " files")


----------------- Word ano-----------------
---- 0 ----
rows: 114
---- 1 ----
rows: 124
---- 2 ----
rows: 122
---- 3 ----
rows: 122
---- 4 ----
rows: 114
---- 5 ----
rows: 92
---- 6 ----
rows: 115
---- 7 ----
rows: 121
---- 8 ----
rows: 118
---- 9 ----
rows: 119
---- 10 ----
rows: 99
---- 11 ----
rows: 125
---- 12 ----
rows: 128
---- 13 ----
rows: 124
---- 14 ----
rows: 128
---- 15 ----
rows: 105
---- 16 ----
rows: 116
---- 17 ----
rows: 106
---- 18 ----
rows: 112
---- 19 ----
rows: 107
---- 20 ----
rows: 102
---- 21 ----
rows: 115
---- 22 ----
rows: 118
---- 23 ----
rows: 130
---- 24 ----
rows: 135
---- 25 ----
rows: 114
---- 26 ----
rows: 133
---- 27 ----
rows: 128
---- 28 ----
rows: 129
---- 29 ----
rows: 122
---- 30 ----
rows: 114
---- 31 ----
rows: 101
---- 32 ----
rows: 99
---- 33 ----
rows: 106
---- 34 ----
rows: 97
---- 35 ----
rows: 120
---- 36 ----
rows: 122
---- 37 ----
rows: 125
---- 38 ----
rows: 121
---- 39 ----
rows: 114
---- 40 ----
rows: 104
---- 41 ----
rows: 93
---- 

---- 40 ----
rows: 128
---- 41 ----
rows: 121
---- 42 ----
rows: 129
---- 43 ----
rows: 105
---- 44 ----
rows: 109
---- 45 ----
rows: 114
---- 46 ----
rows: 107
 NO END ey
---- 47 ----
rows: 98
---- 48 ----
rows: 98
---- 49 ----
 NO END ey
---- 50 ----
rows: 112
---- 51 ----
rows: 119
---- 52 ----
rows: 129
---- 53 ----
rows: 119
---- 54 ----
rows: 134
---- 55 ----
rows: 80
---- 56 ----
rows: 102
---- 57 ----
rows: 91
---- 58 ----
rows: 86
---- 59 ----
rows: 101
---- 60 ----
rows: 96
---- 61 ----
rows: 99
---- 62 ----
rows: 106
---- 63 ----
rows: 89
---- 64 ----
rows: 107
---- 65 ----
rows: 141
---- 66 ----
rows: 148
---- 67 ----
rows: 144
---- 68 ----
rows: 162
---- 69 ----
rows: 149
---- 70 ----
rows: 109
---- 71 ----
rows: 104
---- 72 ----
rows: 108
---- 73 ----
rows: 101
---- 74 ----
rows: 93
---- 75 ----
rows: 113
---- 76 ----
rows: 116
---- 77 ----
rows: 100
---- 78 ----
rows: 102
---- 79 ----
rows: 103
---- 80 ----
rows: 93
---- 81 ----
rows: 97
---- 82 ----
rows: 101
---- 83 --

---- 57 ----
rows: 77
---- 58 ----
rows: 72
---- 59 ----
rows: 62
rows: 84
---- 60 ----
rows: 70
---- 61 ----
rows: 80
---- 62 ----
rows: 80
---- 63 ----
rows: 72
---- 64 ----
rows: 74
 NO END ey
---- 65 ----
rows: 92
---- 66 ----
rows: 85
---- 67 ----
rows: 93
---- 68 ----
rows: 88
---- 69 ----
rows: 96
---- 70 ----
rows: 75
---- 71 ----
rows: 82
---- 72 ----
rows: 78
---- 73 ----
rows: 82
---- 74 ----
rows: 84
---- 75 ----
rows: 77
---- 76 ----
rows: 79
---- 77 ----
rows: 84
---- 78 ----
rows: 87
---- 79 ----
rows: 73
---- 80 ----
rows: 54
rows: 55
rows: 56
rows: 57
rows: 58
rows: 59
rows: 60
rows: 61
rows: 62
rows: 63
rows: 64
rows: 65
rows: 66
rows: 69
rows: 74
 NO END ey
---- 81 ----
rows: 67
rows: 68
rows: 69
rows: 70
---- 82 ----
rows: 76
---- 83 ----
rows: 67
rows: 68
rows: 69
rows: 70
---- 84 ----
rows: 73
---- 85 ----
rows: 87
---- 86 ----
rows: 89
---- 87 ----
rows: 85
---- 88 ----
rows: 99
---- 89 ----
rows: 94
---- 90 ----
rows: 88
---- 91 ----
rows: 93
---- 92 ----
rows: 

---- 81 ----
rows: 84
---- 82 ----
rows: 87
---- 83 ----
rows: 81
---- 84 ----
rows: 76
---- 85 ----
rows: 93
---- 86 ----
rows: 97
---- 87 ----
rows: 100
---- 88 ----
rows: 99
---- 89 ----
rows: 103
---- 90 ----
rows: 90
---- 91 ----
rows: 94
---- 92 ----
rows: 100
---- 93 ----
rows: 97
---- 94 ----
rows: 95
---- 95 ----
rows: 96
---- 96 ----
rows: 90
---- 97 ----
rows: 87
---- 98 ----
rows: 97
---- 99 ----
rows: 98
Done Trimming itim 100 files
----------------- Word ito-----------------
---- 0 ----
rows: 103
---- 1 ----
rows: 102
---- 2 ----
rows: 101
---- 3 ----
rows: 108
---- 4 ----
rows: 111
---- 5 ----
rows: 116
---- 6 ----
rows: 114
---- 7 ----
rows: 114
---- 8 ----
rows: 126
---- 9 ----
rows: 134
---- 10 ----
rows: 37
rows: 50
rows: 103
---- 11 ----
rows: 37
rows: 38
rows: 51
rows: 52
rows: 66
rows: 137
---- 12 ----
rows: 34
rows: 35
rows: 50
rows: 51
rows: 65
rows: 83
---- 13 ----
rows: 36
rows: 52
rows: 115
---- 14 ----
rows: 40
rows: 55
rows: 123
---- 15 ----
rows: 39
rows: 

---- 95 ----
rows: 80
---- 96 ----
rows: 91
---- 97 ----
rows: 97
---- 98 ----
rows: 101
---- 99 ----
rows: 93
Done Trimming kayo 100 files
----------------- Word ko-----------------
---- 0 ----
rows: 78
---- 1 ----
rows: 84
---- 2 ----
rows: 77
---- 3 ----
rows: 79
---- 4 ----
rows: 89
---- 5 ----
rows: 102
---- 6 ----
rows: 120
---- 7 ----
rows: 107
---- 8 ----
rows: 101
---- 9 ----
rows: 113
---- 10 ----
rows: 94
---- 11 ----
rows: 88
---- 12 ----
rows: 98
---- 13 ----
rows: 100
---- 14 ----
rows: 90
---- 15 ----
rows: 82
---- 16 ----
rows: 87
---- 17 ----
rows: 80
---- 18 ----
rows: 91
---- 19 ----
rows: 96
---- 20 ----
rows: 62
rows: 65
rows: 66
rows: 67
rows: 68
rows: 69
rows: 70
---- 21 ----
rows: 69
rows: 70
---- 22 ----
rows: 72
---- 23 ----
rows: 70
---- 24 ----
rows: 70
---- 25 ----
rows: 82
---- 26 ----
rows: 100
---- 27 ----
rows: 103
---- 28 ----
rows: 108
---- 29 ----
rows: 111
---- 30 ----
rows: 52
rows: 53
rows: 54
rows: 55
rows: 56
rows: 57
rows: 58
rows: 59
rows: 60


...

rows: 68
rows: 69
rows: 70
---- 95 ----
rows: 80
---- 96 ----
rows: 79
---- 97 ----
rows: 82
---- 98 ----
rows: 93
---- 99 ----
rows: 80
Done Trimming magkano 100 files
----------------- Word manok-----------------
---- 0 ----
rows: 109
---- 1 ----
rows: 124
---- 2 ----
rows: 111
---- 3 ----
rows: 123
---- 4 ----
rows: 119
---- 5 ----
rows: 125
---- 6 ----
rows: 133
---- 7 ----
rows: 135
---- 8 ----
rows: 129
---- 9 ----
rows: 125
---- 10 ----
rows: 122
---- 11 ----
rows: 128
---- 12 ----
rows: 121
---- 13 ----
rows: 143
---- 14 ----
rows: 116
---- 15 ----
rows: 108
---- 16 ----
rows: 103
---- 17 ----
rows: 111
---- 18 ----
rows: 114
---- 19 ----
rows: 120
---- 20 ----
rows: 99
---- 21 ----
rows: 113
---- 22 ----
rows: 96
---- 23 ----
rows: 112
---- 24 ----
rows: 102
---- 25 ----
rows: 100
---- 26 ----
rows: 101
---- 27 ----
rows: 121
---- 28 ----
rows: 134
---- 29 ----
rows: 124
---- 30 ----
rows: 99
---- 31 ----
rows: 98
---- 32 ----
rows: 96
---- 33 ----
rows: 94
---- 34 ----
rows: 

---- 30 ----
rows: 84
---- 31 ----
rows: 88
---- 32 ----
rows: 90
---- 33 ----
rows: 87
---- 34 ----
rows: 80
---- 35 ----
rows: 76
---- 36 ----
rows: 81
---- 37 ----
rows: 76
---- 38 ----
rows: 68
rows: 69
rows: 70
---- 39 ----
rows: 81
---- 40 ----
rows: 80
---- 41 ----
rows: 100
---- 42 ----
rows: 96
---- 43 ----
rows: 91
---- 44 ----
rows: 92
---- 45 ----
rows: 87
---- 46 ----
rows: 90
---- 47 ----
rows: 90
---- 48 ----
rows: 83
---- 49 ----
rows: 84
---- 50 ----
rows: 111
---- 51 ----
rows: 111
---- 52 ----
rows: 115
---- 53 ----
rows: 116
---- 54 ----
rows: 116
---- 55 ----
rows: 87
---- 56 ----
rows: 86
---- 57 ----
rows: 80
---- 58 ----
rows: 88
---- 59 ----
rows: 79
---- 60 ----
rows: 94
---- 61 ----
rows: 90
---- 62 ----
rows: 94
---- 63 ----
rows: 104
---- 64 ----
rows: 100
---- 65 ----
rows: 100
---- 66 ----
rows: 114
---- 67 ----
rows: 109
---- 68 ----
rows: 94
---- 69 ----
rows: 109
---- 70 ----
rows: 82
---- 71 ----
rows: 83
---- 72 ----
rows: 82
---- 73 ----
rows: 91
--

---- 2 ----
rows: 99
---- 3 ----
rows: 105
---- 4 ----
rows: 107
---- 5 ----
rows: 107
---- 6 ----
rows: 109
---- 7 ----
rows: 114
---- 8 ----
rows: 112
---- 9 ----
rows: 112
---- 10 ----
rows: 107
---- 11 ----
rows: 109
---- 12 ----
rows: 112
---- 13 ----
rows: 110
---- 14 ----
rows: 106
---- 15 ----
rows: 93
---- 16 ----
rows: 92
---- 17 ----
rows: 96
---- 18 ----
rows: 93
---- 19 ----
rows: 94
---- 20 ----
rows: 82
---- 21 ----
rows: 92
---- 22 ----
rows: 92
---- 23 ----
rows: 83
---- 24 ----
rows: 89
---- 25 ----
rows: 84
---- 26 ----
rows: 95
---- 27 ----
rows: 102
---- 28 ----
rows: 94
---- 29 ----
rows: 103
---- 30 ----
rows: 72
---- 31 ----
rows: 69
rows: 70
---- 32 ----
rows: 70
---- 33 ----
rows: 74
---- 34 ----
rows: 67
rows: 69
rows: 70
---- 35 ----
rows: 75
---- 36 ----
rows: 85
---- 37 ----
rows: 73
---- 38 ----
rows: 82
---- 39 ----
rows: 90
---- 40 ----
rows: 95
---- 41 ----
rows: 78
---- 42 ----
rows: 77
---- 43 ----
rows: 88
---- 44 ----
rows: 70
---- 45 ----
rows: 72

---- 24 ----
rows: 105
---- 25 ----
rows: 99
---- 26 ----
rows: 100
---- 27 ----
rows: 110
---- 28 ----
rows: 97
---- 29 ----
rows: 101
---- 30 ----
rows: 86
---- 31 ----
rows: 83
---- 32 ----
rows: 84
---- 33 ----
rows: 87
---- 34 ----
rows: 87
---- 35 ----
rows: 97
---- 36 ----
rows: 109
---- 37 ----
rows: 118
---- 38 ----
rows: 106
---- 39 ----
rows: 97
---- 40 ----
rows: 77
---- 41 ----
rows: 69
rows: 70
---- 42 ----
rows: 84
---- 43 ----
rows: 80
---- 44 ----
rows: 85
---- 45 ----
rows: 85
---- 46 ----
rows: 99
---- 47 ----
rows: 97
---- 48 ----
rows: 95
---- 49 ----
rows: 111
---- 50 ----
rows: 113
---- 51 ----
rows: 113
---- 52 ----
rows: 110
---- 53 ----
rows: 112
---- 54 ----
rows: 121
---- 55 ----
rows: 90
---- 56 ----
rows: 82
---- 57 ----
rows: 103
---- 58 ----
rows: 93
---- 59 ----
rows: 87
---- 60 ----
rows: 120
---- 61 ----
rows: 92
---- 62 ----
rows: 120
---- 63 ----
rows: 113
 NO END ey
---- 64 ----
rows: 97
 NO END ey
---- 65 ----
rows: 122
---- 66 ----
rows: 129
----

rows: 82
---- 58 ----
rows: 71
---- 59 ----
rows: 74
---- 60 ----
rows: 63
rows: 64
rows: 66
rows: 67
rows: 68
rows: 69
rows: 74
---- 61 ----
rows: 75
---- 62 ----
rows: 74
---- 63 ----
rows: 84
---- 64 ----
rows: 71
---- 65 ----
rows: 93
---- 66 ----
rows: 114
---- 67 ----
rows: 110
---- 68 ----
rows: 109
---- 69 ----
rows: 115
---- 70 ----
rows: 69
rows: 70
---- 71 ----
rows: 73
---- 72 ----
rows: 81
---- 73 ----
rows: 81
---- 74 ----
rows: 77
---- 75 ----
rows: 83
---- 76 ----
rows: 88
---- 77 ----
rows: 84
---- 78 ----
rows: 78
---- 79 ----
rows: 84
---- 80 ----
rows: 80
---- 81 ----
rows: 77
---- 82 ----
rows: 82
---- 83 ----
rows: 80
---- 84 ----
rows: 84
---- 85 ----
rows: 90
---- 86 ----
rows: 92
---- 87 ----
rows: 98
---- 88 ----
rows: 85
---- 89 ----
rows: 98
---- 90 ----
rows: 86
---- 91 ----
rows: 88
---- 92 ----
rows: 92
---- 93 ----
rows: 91
---- 94 ----
rows: 94
---- 95 ----
rows: 100
---- 96 ----
rows: 102
---- 97 ----
rows: 104
---- 98 ----
rows: 102
---- 99 ----
rows:

In [ ]:
end

In [6]:
import os

directory = "trimmed_data_updated/"
if not os.path.exists(directory):
    os.makedirs(directory)
    
for className in classNames:
    className = className.split("\\")[1]
    
    if not os.path.exists(directory +className):
        os.makedirs(directory  +className)
    
    ctr = 0
    for df in listOfTrimmedDataFramesPerClass[className]['emg']:
        fileName = directory + className + "/emg" + str(ctr)
        df['right'].to_csv(fileName + "-Right2a.csv", encoding='utf-8', index=False)
        df['left'].to_csv(fileName + "-Left2a.csv", encoding='utf-8', index=False)
        ctr+=1
    
    ctr = 0
    for df in listOfTrimmedDataFramesPerClass[className]['acc']:
        fileName = directory + className + "/acc" + str(ctr)
        df['right'].to_csv(fileName + "-Right2a.csv", encoding='utf-8', index=False)
        df['left'].to_csv(fileName + "-Left2a.csv", encoding='utf-8', index=False)
        ctr+=1

    ctr = 0
    for df in listOfTrimmedDataFramesPerClass[className]['gyro']:
        fileName = directory + className + "/gyro" + str(ctr)
        df['right'].to_csv(fileName + "-Right2a.csv", encoding='utf-8', index=False)
        df['left'].to_csv(fileName + "-Left2a.csv", encoding='utf-8', index=False)
        ctr+=1
    
    print("Done saving trimmed files of " + className)

print("DONE")

Done saving trimmed files of ano
Done saving trimmed files of asul
Done saving trimmed files of ate
Done saving trimmed files of baboy
Done saving trimmed files of dalawa
Done saving trimmed files of dito
Done saving trimmed files of hello
Done saving trimmed files of isa
Done saving trimmed files of isda
Done saving trimmed files of itim
Done saving trimmed files of ito
Done saving trimmed files of kanin
Done saving trimmed files of kayo
Done saving trimmed files of ko
Done saving trimmed files of kulay
Done saving trimmed files of kuya
Done saving trimmed files of large (shirt)
Done saving trimmed files of magkano
Done saving trimmed files of manok
Done saving trimmed files of mantika
Done saving trimmed files of medium (shirt)
Done saving trimmed files of nito
Done saving trimmed files of niyo
Done saving trimmed files of pagkain
Done saving trimmed files of pula
Done saving trimmed files of salamat
Done saving trimmed files of small (shirt)
Done saving trimmed files of softdrinks
D

In [ ]:
# listOfTrimmedDataFramesPerClass = {}

# for className in classNames:
#     className = className.split("\\")[1]
#     print("----------------- Word " + className + "-----------------")
#     ctr = 0
#     trimmedDataFramesPerClass = {}
#     trimmedDataFramesPerClass['left'] = []
#     trimmedDataFramesPerClass['right'] = []
    
#     dataFrameDictionary = {}
    
#     emgDataFrames = listOfDataFramesPerClass[className]['emg']
#     accDataFrames = listOfDataFramesPerClass[className]['acc']
#     gyroDataFrames = listOfDataFramesPerClass[className]['gyro']
    
#     trimmedAccDataFrames = []
#     trimmedGyroDataFrames = []
#     trimmedEmgDataFrames = []

    
#     for i in range(0,len(listOfDataFramesPerClass[className]['acc']['right'])):
#         start = -1
#         end = -1
#         counter = 0
#         print("---- %s ----" % i)
#         ctr += 1
#         dataFrame = listOfDataFramesPerClass[className]['acc']['right'][i]
#         for x, z in zip(dataFrame['x'], dataFrame['z']):  
#             noend = False
#             isBreak = False
#             if(className in ("isda", "magkano") and i in range(60,65)):
#                 if(x < 0.85 and start ==-1):
#                     #print(x)
#                     start = counter - 10
#                     if start < 0:
#                         start = 0
#                 elif(start != -1 and x>0.85):
                    
#                     end = counter + 10
#                     if(start-end < 45):
#                         end = -1
#                        # print("EYY")
#                     else:
#                         isBreak = True
                        
#                 if(isBreak =):
#                     noend = False
#                     break;
#             else:
#                 if(x < z and start ==-1):
#                     #print(x)
#                     start = counter - 10
#                     if start < 0:
#                         start = 0
#                 elif(start != -1 and x>0.85):
#                     end = counter + 10
#                     if(start-end < 45):
#                         end = -1
#                        # print("EYY")
#                     else:
#                         isBreak = True
                        
                
#                 noend = True
                
#                 if(isBreak):
#                     noend = False
#                     break;
#             counter += 1
            
#         if noend == True or end >= len(listOfDataFramesPerClass[className]['acc']['right'][i]['x']):
#             print(" NO END ey" )
#             end = len(listOfDataFramesPerClass[className]['acc']['right'][i]['x'])-1
        

#         startEmg = -1
#         endEmg = -1
#         counter2 = 0
        
#         #print("Start index: %s End index: %s" % (start, end))
#         for t in emgDataFrames['right'][i]['timestamp']:
#             noend = False
#             if(t > accDataFrames['right'][i].loc[start]['timestamp'] and startEmg == -1):
#                 #print("%s > %f" % (t, accDataFrames['right'][i].loc[start]['timestamp']))
#                 startEmg = counter2
#             elif(startEmg != -1 and t > accDataFrames['right'][i].loc[end]['timestamp']):
#                 #print("%s > %f" % (t, accDataFrames['right'][i].loc[end]['timestamp']))
#                 endEmg = counter2
#                 break;
#             else:
#                 noend = True
#             counter2 += 1
        
#         if noend == True:
#        #     print(" NO END")
#             endEmg = len(listOfDataFramesPerClass[className]['emg']['right'][i]['timestamp'])-1
        
#         newAccDf = {}
#         newAccDf['right'] = accDataFrames['right'][i].loc[start:end]
#         newAccDf['right'] = newAccDf['right'].reset_index(drop=True)
#         newAccDf['left'] = accDataFrames['left'][i].loc[start:end]
#         newAccDf['left'] = newAccDf['left'].reset_index(drop=True)
        
#         newGyroDf = {}
#         newGyroDf['right'] = gyroDataFrames['right'][i].loc[start:end]
#         newGyroDf['right'] = newGyroDf['right'].reset_index(drop=True)
#         newGyroDf['left'] = gyroDataFrames['left'][i].loc[start:end]
#         newGyroDf['left'] = newGyroDf['left'].reset_index(drop=True)
        
#         newEmgDf = {}
#         newEmgDf['right'] = emgDataFrames['right'][i].loc[startEmg:endEmg]
#         newEmgDf['right'] = newEmgDf['right'].reset_index(drop=True)
#         newEmgDf['left'] = emgDataFrames['left'][i].loc[startEmg:endEmg]
#         newEmgDf['left'] = newEmgDf['left'].reset_index(drop=True)

#         trimmedAccDataFrames.append(newAccDf)
#         trimmedGyroDataFrames.append(newGyroDf)
#         trimmedEmgDataFrames.append(newEmgDf)
        
#         #print("Start emg index: %s End emg index: %s" % (startEmg, endEmg))
        
#     dataFrameDictionary['acc'] = trimmedAccDataFrames
#     dataFrameDictionary['gyro'] = trimmedGyroDataFrames
#     dataFrameDictionary['emg'] = trimmedEmgDataFrames
        
#     listOfTrimmedDataFramesPerClass[className] = dataFrameDictionary

        
    
#     print("Done Trimming " + className + " " + str(ctr) + " files")


In [ ]:
# trimmedAccDataFrames = {}
# trimmedAccDataFrames['right'] = []
# trimmedAccDataFrames['left'] = []

# trimmedGyroDataFrames = {}
# trimmedGyroDataFrames['right'] = []
# trimmedGyroDataFrames['left'] = []

# trimmedEmgDataFrames = {}
# trimmedEmgDataFrames['right'] = []
# trimmedEmgDataFrames['left'] = []

# for i in range(0,len(gyroDataFrames['right'])):
#     start = -1
#     end = -1
#     counter = 0
#     for x in accDataFrames['right'][i]['x']:  
#         if(x < 0.85 and start ==-1):
#             print(x)
#             start = counter - 10
#         elif(start != -1 and x>0.99):
#             end = counter + 9
#             break;
#         counter += 1
    
#     startEmg = -1
#     endEmg = -1
#     counter2 = 0
#     for t in emgDataFrames['right'][i]['timestamp']:
#         if(t > accDataFrames['right'][i].loc[start]['timestamp'] and startEmg == -1):
#             #print("%s > %f" % (t, accDataFrames['right'][i].loc[start]['timestamp']))
#             startEmg = counter2-1
#         elif(startEmg != -1 and t > accDataFrames['right'][i].loc[end]['timestamp']):
#             #print("%s > %f" % (t, accDataFrames['right'][i].loc[end]['timestamp']))
#             endEmg = counter2-1
#             break;
#         counter2 += 1
    
#     newAccDf = accDataFrames['right'][i].loc[start:end]
#     newAccDf = newAccDf.reset_index(drop=True)
#     newGyroDf = gyroDataFrames['right'][i].loc[start:end]
#     newGyroDf = newGyroDf.reset_index(drop=True)
#     newEmgDf = emgDataFrames['right'][i].loc[startEmg:endEmg]
#     newEmgDf = newEmgDf.reset_index(drop=True)

#     trimmedAccDataFrames['right'].append(newAccDf)
#     trimmedGyroDataFrames['right'].append(newGyroDf)
#     trimmedEmgDataFrames['right'].append(newEmgDf)
    
#     print("Start index: %s End index: %s" % (start, end))
#     print("Start emg index: %s End emg index: %s" % (startEmg, endEmg))


In [ ]:
# import glob
# import pandas as pd

# listOfFileNames = []
# listOfClassNames = []
# listOfDataPerClass = []
# files = []
# classNames = []

# files = glob.glob("/Users/user/Desktop/FSL by non-signers/ano/1/*.csv")
# print(len(files))
# listOfFileNames = []
# for f in files:
#     listOfFileNames.append(f.split("\\")[1])
    
# emgDataFrames = {}
# emgDataFrames['right'] = []
# emgDataFrames['left'] = []
    
# accDataFrames = {}
# accDataFrames['right'] = []
# accDataFrames['left'] = []

# gyroDataFrames = {}
# gyroDataFrames['right'] = []
# gyroDataFrames['left'] = []

# dataFrameDictionary = {}
# leftctr = 0
# rightctr = 0
# for fileName in listOfFileNames:
#     fileLocation = "/Users/user/Desktop/FSL by non-signers/ano/1/"+fileName 
#     #print(fileLocation)
#     df = pd.read_csv(fileLocation)
    
#     if "emg" in fileName:   
#         if "-0-" in fileName:
#             emgDataFrames['right'].append(df)
#             leftctr+=1
#         else:
#             emgDataFrames['left'].append(df)
#             rightctr+=1
#     elif ("accelerometer") in fileName:
#         if "-0-" in fileName:
#             accDataFrames['right'].append(df)
#             leftctr+=1
#         else:
#             accDataFrames['left'].append(df)
#             rightctr+=1
#     elif ("gyro") in fileName:
#         if "-0-" in fileName:
#             gyroDataFrames['right'].append(df)
#             leftctr+=1
#         else:
#             gyroDataFrames['left'].append(df)      
#             rightctr+=1

# print("Total of " + str(leftctr) + " left arm files and " + str(rightctr) + " right arm files.")

    
# #listOfClassNames
# #listOfDataPerClass[0]
# print("Done listing the class and file names")

# Pancheck if goods yung trim

In [ ]:
# import matplotlib.pyplot as plt

# for i in range(0,len(trimmedAccDataFrames['right'])):
#     print("-----%s------" % str(i+1))
#     print("Untrimmed")

#     fig_size = plt.rcParams["figure.figsize"]    
#     fig_size[0] = 14
#     fig_size[1] = 4
#     plt.rcParams["figure.figsize"] = fig_size

#     plt.plot(accDataFrames['right'][i]['x'])
#     plt.plot(accDataFrames['right'][i]['y'])
#     plt.plot(accDataFrames['right'][i]['z'])
    
# #     plt.plot(emgDataFrames['right'][i]['emg1'])
# #     plt.plot(emgDataFrames['right'][i]['emg2'])
# #     plt.plot(emgDataFrames['right'][i]['emg3'])
# #     plt.plot(emgDataFrames['right'][i]['emg4'])
# #     plt.plot(emgDataFrames['right'][i]['emg5'])
# #     plt.plot(emgDataFrames['right'][i]['emg6'])
# #     plt.plot(emgDataFrames['right'][i]['emg7'])
# #     plt.plot(emgDataFrames['right'][i]['emg8'])
    
# #     plt.plot(gyroDataFrames['right'][i]['x'])
# #     plt.plot(gyroDataFrames['right'][i]['y'])
# #     plt.plot(gyroDataFrames['right'][i]['z'])
#     plt.show()
#     print("Trimmed")

#     fig_size = plt.rcParams["figure.figsize"]    
#     fig_size[0] = 14
#     fig_size[1] = 4
#     plt.rcParams["figure.figsize"] = fig_size
#     plt.plot(trimmedAccDataFrames['right'][i]['x'])
#     plt.plot(trimmedAccDataFrames['right'][i]['y'])
#     plt.plot(trimmedAccDataFrames['right'][i]['z'])
    
# #     plt.plot(trimmedEmgDataFrames['right'][i]['emg1'])
# #     plt.plot(trimmedEmgDataFrames['right'][i]['emg2'])
# #     plt.plot(trimmedEmgDataFrames['right'][i]['emg3'])
# #     plt.plot(trimmedEmgDataFrames['right'][i]['emg4'])
# #     plt.plot(trimmedEmgDataFrames['right'][i]['emg5'])
# #     plt.plot(trimmedEmgDataFrames['right'][i]['emg6'])
# #     plt.plot(trimmedEmgDataFrames['right'][i]['emg7'])
# #     plt.plot(trimmedEmgDataFrames['right'][i]['emg8'])

# #     plt.plot(trimmedGyroDataFrames['right'][i]['x'])
# #     plt.plot(trimmedGyroDataFrames['right'][i]['y'])
# #     plt.plot(trimmedGyroDataFrames['right'][i]['z'])
#     plt.show()

In [ ]:
# import numpy as np

# start = -1
# end = -1
# counter = 0
# for x in accDataFrames['right'][1]['x']:  
#     mean = np.mean(accDataFrames['right'][0]['x'].loc[0:counter])
#     sd = np.std(accDataFrames['right'][0]['x'].loc[0:counter])*4
#     print("n: %s mean: %s sd: %s mean+sd = %s" % (accDataFrames['right'][0]['x'][counter], mean, sd, mean+sd))
    
#     if(counter>0):
#         if(accDataFrames['right'][1]['y'][counter+1] < (mean-sd)):
#             print(counter+1)
#             break;
#     counter +=1
    


# Save file